In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import types

In [138]:
ubatch = torch.rand(8, 197, 768)
ubatch_labels = torch.tensor([3, 13, 15, 20, 40, 285, 285, 285])

In [139]:
classifier = nn.Linear(768, 1000)

In [140]:
classifier(ubatch)

tensor([[[ 0.2766,  0.0448,  0.0932,  ..., -0.0393,  0.5249, -0.0993],
         [-0.0343, -0.2425,  0.1282,  ...,  0.3778,  0.2658,  0.2387],
         [ 0.1523, -0.4120,  0.0839,  ..., -0.1985,  0.1147, -0.2054],
         ...,
         [ 0.0630, -0.1931,  0.0755,  ...,  0.0412,  0.1973, -0.1324],
         [ 0.1684, -0.3691,  0.1220,  ...,  0.1200,  0.3867,  0.0300],
         [ 0.0945, -0.2799,  0.2160,  ...,  0.3509,  0.5158, -0.1098]],

        [[-0.0284, -0.1280, -0.1112,  ...,  0.3189,  0.0875,  0.1434],
         [-0.3748, -0.3916,  0.0476,  ...,  0.0858,  0.1880, -0.0721],
         [ 0.0698, -0.4706,  0.1800,  ..., -0.0121,  0.3323, -0.3474],
         ...,
         [-0.2075, -0.4918,  0.1257,  ...,  0.3077,  0.5018, -0.2906],
         [ 0.0824, -0.4345,  0.1456,  ..., -0.0886,  0.5073,  0.3248],
         [ 0.0635, -0.2664,  0.2375,  ...,  0.2204,  0.3783,  0.1531]],

        [[ 0.1382, -0.2284,  0.0519,  ..., -0.1145,  0.5667,  0.0708],
         [ 0.3807, -0.1844,  0.1266,  ..., -0

In [141]:
class ModuleShard(nn.Module):
    def __init__(self):
        super().__init__()

class VitImageClassification(ModuleShard):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Linear(768, 1000)
    
def snip_forward_linear(self, x):
    return F.linear(x, self.weight * self.weight_mask, self.bias)

In [142]:
vit = VitImageClassification()
vit.classifier.weight_mask = nn.Parameter(torch.ones_like(vit.classifier.weight))
vit.classifier.forward = types.MethodType(snip_forward_linear, vit.classifier)
vit.zero_grad()
output = vit.classifier(ubatch[:,0,:])
loss = F.nll_loss(output, ubatch_labels)
loss.backward()

In [166]:
grads_abs = []
grads_abs.append(torch.abs(vit.classifier.weight_mask.grad))
# grads_abs.append(torch.abs(a))
all_scores = torch.cat([torch.flatten(x) for x in grads_abs])
norm_factor = torch.sum(all_scores)
all_scores.div_(norm_factor)
keep_ratio = 0.0001
num_params_to_keep = int(len(all_scores) * keep_ratio)
threshold, _ = torch.topk(all_scores, num_params_to_keep, sorted=True)
acceptable_score = threshold[-1]
keep_masks = []
for g in grads_abs:
    keep_masks.append(((g / norm_factor) >= acceptable_score).float())
print(torch.sum(torch.cat([torch.flatten(x == 1) for x in keep_masks])))

tensor(76)


In [119]:
grads_abs = []
grads_abs.append(torch.abs(vit.classifier.weight_mask.grad))
grads_abs.append(torch.abs(vit.classifier.weight_mask.grad))
all_scores = torch.cat([torch.flatten(x) for x in grads_abs])

In [171]:
keep_masks[0][keep_masks[0] == 1]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.])

In [170]:
vit.classifier.weight.data[keep_masks[0] == 0.].shape

torch.Size([767924])

In [164]:
for keep_mask in zip(keep_masks):
    print(keep_mask)
    keep_mask.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]),)


AttributeError: 'tuple' object has no attribute 'shape'

In [172]:
vit.classifier.weight.data[keep_masks[0] == 0.] = 0.0

tensor(768000)
